In [3]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
plt.rcParams['figure.max_open_warning'] = 0
import numpy as np
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

# for Box-Cox Transformation if I do it
#from scipy import stats

# for min_max scaling
from mlxtend.preprocessing import minmax_scaling

df3 = pd.read_table(r"D:\Yuliya_test_data_table\Yuliya_test_data_table_deconv-only_3prime.txt", delimiter = '\t')#, nrows=2)
df5 = pd.read_table(r"D:\Yuliya_test_data_table\Yuliya_test_data_table_deconv-only.txt", delimiter = '\t')#, nrows=2)


# Process Data

In [42]:
dfclean5 = df5.copy()
#keep this order because I went through the output file individually to determine these lines
dfclean5 = dfclean5[~dfclean5.index.isin(range(33482, 33797))] #122
dfclean5 = dfclean5[~dfclean5.index.isin(range(34747, 35652))] #126, 127, 128
dfclean5 = dfclean5[~dfclean5.index.isin(range(35930, 36501))] #130, 131
dfclean5 = dfclean5[~dfclean5.index.isin(range(36770, 37054))] #133
dfclean5 = dfclean5[~dfclean5.index.isin(range(38773, 39015))] #140
dfclean5 = dfclean5[~dfclean5.index.isin(range(39325, 39543))] #142
dfclean5 = dfclean5[~dfclean5.index.isin(range(40030, 40626))] #145, 146
dfclean5 = dfclean5[~dfclean5.index.isin(range(46853, 49718))] #170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182
dfclean5 = dfclean5[~(dfclean5.index >= 115849)] #the rest


In [43]:
dfclean5_scaled = dfclean5
#dfclean5adj_scaled = dfclean5_adjustedto_m
columns_to_standardize = ['C15', 'C25', 'D15', 'D25', 'R15', 'R25']
dfclean5_scaled[columns_to_standardize] = scaler.fit_transform(dfclean5[columns_to_standardize])
#dfclean5adj_scaled[columns_to_standardize] = scaler.fit_transform(dfclean5_adjustedto_m[columns_to_standardize])

In [49]:
droppedR_df = dfclean5_scaled
droppedR_df['R_diff'] =abs(droppedR_df['R15']-droppedR_df['R25'])
droppedR_df['plates'] = droppedR_df['plate'].str.extract(r'(\d+)').astype(int)

droppedR_df = droppedR_df[droppedR_df['R_diff'] <0.99]
#print(droppedR_df)

# Compare C to R

In [45]:
dfclean5_scaled['plates'] = dfclean5_scaled['plate'].str.extract(r'(\d+)').astype(int)

C_df_R1_only = dfclean5_scaled[['IB', 'plates', 'well', 'C15', 'C25', 'R15']]
C_df_R1_only['C15Difference'] = C_df_R1_only['R15'] - C_df_R1_only['C15']
C_df_R1_only['C25Difference'] = C_df_R1_only['R15'] - C_df_R1_only['C25']


C_df_droppedR = droppedR_df[['IB', 'plates', 'well', 'C15', 'C25', 'R15', 'R25', 'R_diff']]
C_df_droppedR['Ravg'] = (C_df_droppedR['R15']+C_df_droppedR['R25'])/2
C_df_droppedR['C15Difference'] = (C_df_droppedR['R15']+C_df_droppedR['R25']/2) - C_df_droppedR['C15']
C_df_droppedR['C25Difference'] = ((C_df_droppedR['R15']+C_df_droppedR['R25'])/2) - C_df_droppedR['C25']


print(C_df_R1_only.columns)

Index(['IB', 'plates', 'well', 'C15', 'C25', 'R15', 'C15Difference',
       'C25Difference'],
      dtype='object')


C:\Users\Administrator\AppData\Local\Temp\ipykernel_15564\246555905.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  C_df_R1_only['C15Difference'] = C_df_R1_only['R15'] - C_df_R1_only['C15']
C:\Users\Administrator\AppData\Local\Temp\ipykernel_15564\246555905.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  C_df_R1_only['C25Difference'] = C_df_R1_only['R15'] - C_df_R1_only['C25']
C:\Users\Administrator\AppData\Local\Temp\ipykernel_15564\246555905.py:9: SettingWithCopyWarning: 
A value is trying to be s

In [46]:
C_df_R1_onlyg= {}

for pattern, group in C_df_R1_only.groupby('plates'):
    C_df_R1_onlyg[pattern] = group
    #print( group.columns)

C_df_droppedRg= {}

for pattern, group in C_df_droppedR.groupby('plates'):
    C_df_droppedRg[pattern] = group
    #print( group.columns)

In [1]:
for key, df in C_df_R1_onlyg.items():
    plt.figure(figsize=(25, 6))
    plt.scatter(df.index, df['C15'].astype(int), label='C15')  
    plt.scatter(df.index, df['C25'].astype(int), label='C25')  
    plt.scatter(df.index, df['R15'].astype(int), label='R15')  
    #plt.scatter(df.index, df['C15Difference'].astype(int), label='Dif. to R15')  # Convert to int and plot 'C25' column

    
    plt.xlabel('Index')  # Assuming 'well' is not a numeric column, otherwise, replace with 'well'
    plt.ylabel('Data')
    plt.title(f'Plot for plate {key}')
    plt.legend()
    plt.show()

NameError: name 'C_df_R1_onlyg' is not defined

In [2]:
for key, df in C_df_droppedRg.items():
    plt.figure(figsize=(25, 6))
    plt.scatter(df.index, df['C15'].astype(int), label='C15') 
    #plt.scatter(df.index, df['C25'].astype(int), label='C25')  # Convert to int and plot 'C15' column
    plt.scatter(df.index, df['Ravg'].astype(int), label='Ravg')  # Convert to int and plot 'C25' column
   # plt.scatter(df.index, df['R25'].astype(int), label='R25')  # Convert to int and plot 'C25' column
    #plt.scatter(df.index, df['C15Difference'].astype(int), label='Difference to R')  # Convert to int and plot 'C25' column

    
    plt.xlabel('Index')  # Assuming 'well' is not a numeric column, otherwise, replace with 'well'
    plt.ylabel('Data')
    plt.title(f'Plot for plate {key}')
    plt.legend()
    plt.show()

NameError: name 'C_df_droppedRg' is not defined

In [ ]:
for key, df in C_df_R1_onlyg.items():
    plt.figure(figsize=(10, 6))
    plt.boxplot([df['differencetoR1']])
    plt.title(f'Difference range for plate {key}')

plt.show()

In [ ]:
for key, df in C_df_droppedRg.items():
    plt.figure(figsize=(10, 6))
    plt.boxplot([df['differencetoR1']])
    plt.title(f'Difference range for plate {key}')

plt.show()

# Compare C to C

# Compare D to R

# Compare D to D

# old to new